In [1]:
import pandas as pd
import numpy as np
from R32_bmimPF6_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.07
    init_kappa_1_2A = -0.02
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R32'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimPF6'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R32'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimPF6','bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_B['bmimPF6', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'bmimPF6'].fix(0)
    m.fs.properties.PR_kappa_C['bmimPF6', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'bmimPF6'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()
    m.fs.state_block.mole_frac_comp['R32'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data['x_R32']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data['x_bmimPF6']))
    m.fs.state_block.mole_frac_comp['R32'].fix(float(data['x_R32'])+eps)
    m.fs.state_block.mole_frac_comp['bmimPF6'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(5)

    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-5)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimPF6', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'bmimPF6']"]

#     pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

#     obj_value, parameters,a= pest.theta_est(calc_cov=True)

In [3]:
data = pd.read_csv('R32IL.csv')

# print(data)


In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.09 #mole fraction

for i in range(N):
    print(i)

    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
    data_["x_bmimPF6"] += data_["x_bmimPF6"] * noise_x

    data_.loc[data_['x_bmimPF6'] <= 0.0001, 'x_bmimPF6'] = 0.0001
    data_.loc[data_['x_bmimPF6'] >= 0.9999 , 'x_bmimPF6'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.19072722146728807
2022-03-04 20:27:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:27:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04

2022-03-04 20:28:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:28:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:28:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:28:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:28:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:28:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:28:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:28:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:29:01 [INFO] idaes

2022-03-04 20:30:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:30:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:30:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:30:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:30:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:30:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:30:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:30:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:30:23 [INFO] idaes

2022-03-04 20:31:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:31:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:31:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:31:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:31:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:31:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:31:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:31:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:31:48 [INFO] idaes

2022-03-04 20:33:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 20:33:20 [INFO] idaes

...

2022-03-04 21:25:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:25:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:25:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:25:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:25:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:25:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:25:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:25:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:25:59 [INFO] idaes

2022-03-04 21:27:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:27:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:27:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:27:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:27:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:27:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:27:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:27:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimPF6

2022-03-04 21:28:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:28:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:28:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:28:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:28:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:28:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimPF6,R32] = -0.04944815114168081
fs.properties.PR_kappa_A[R32,bmimPF6] = -0.02723805050748176
95
-0.023866914651840102
2022-03-04 21:28:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-0

2022-03-04 21:30:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:30:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimPF6,R32] = -0.09134840564990863
fs.properties.PR_kappa_A[R32,bmimPF6] = -0.028229767061554972
97
-0.08099003674389149
2022-03-04 21:30:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:30:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:30:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:30:17 [INFO] idaes.init.fs.state_block: Prope

fs.properties.PR_kappa_A[bmimPF6,R32] = -0.048677072147479225
fs.properties.PR_kappa_A[R32,bmimPF6] = -0.02720892075374511
99
-0.11038537497251927
2022-03-04 21:31:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:31:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:31:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:31:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:31:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:31:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-04 21:31:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-0

In [5]:
print(results_params)

[[-0.09433049 -0.02843036]
 [-0.07168091 -0.0265657 ]
 [-0.09252159 -0.0283574 ]
 [-0.09382202 -0.028479  ]
 [-0.09016498 -0.02809251]
 [-0.08606771 -0.02741877]
 [-0.03426015 -0.02663517]
 [-0.09105998 -0.02819914]
 [-0.0881345  -0.0278003 ]
 [-0.06118125 -0.02766404]
 [-0.08964959 -0.02801761]
 [-0.05783197 -0.02754562]
 [-0.05649339 -0.0274976 ]
 [-0.05357462 -0.02739149]
 [-0.02598888 -0.02626991]
 [-0.08736751 -0.027683  ]
 [-0.02585962 -0.02626807]
 [-0.04049912 -0.02689055]
 [-0.07094034 -0.02799569]
 [-0.05263974 -0.02735708]
 [-0.05102178 -0.02729705]
 [-0.09322322 -0.02843359]
 [-0.08745359 -0.02766265]
 [-0.09167918 -0.02826665]
 [-0.02596723 -0.02626959]
 [-0.0656246  -0.02781744]
 [-0.03503175 -0.02666738]
 [-0.08748275 -0.02769874]
 [-0.09312956 -0.02842458]
 [-0.08851051 -0.02784474]
 [-0.08772048 -0.02769593]
 [-0.08785686 -0.02775366]
 [-0.06294616 -0.02772547]
 [-0.08766994 -0.02773454]
 [-0.07168149 -0.02656592]
 [-0.02600221 -0.02627012]
 [-0.08461151 -0.0273185 ]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R32_bmimPF6_09_2.csv')